In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# work pipeline
1. [Get the data](#s1)
1. [Explore and visualize the data to gain insight](#s2)
1. [prepare the data](#s3)
1. [select and train a model](#s4)
    1. [LinearRegression](#ln)
    1. [RandomForestRegressor](#rf)
    1. [GradientBoostingRegressor](#GBR)
1. [fine tune and evaluate the model](#s5)
    1. [fine tune RandomForestRegressor](#fnrf)
    1. [GradientBoostingRegressor](#fnGBR)
1. [submit the result](#s6)

# Get the data
<a id="s1"></a>

download the data

In [ ]:
data = pd.read_csv('/kaggle/input/diamond-price-prediciton-2024/train.csv')
submit_data = pd.read_csv('/kaggle/input/diamond-price-prediciton-2024/test.csv')

In [ ]:
data.shape

we need to get testset befor doing any thing 

split the data to 80% trainset and 20% testset 

In [ ]:
from sklearn.model_selection import train_test_split
train , test = train_test_split(data , test_size=0.2, random_state=42)

# Explore and visualize the data to gain insight
<a id="s2"></a>

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.describe(include = object)

from describe table we can see that the min value in the columns x,y,z  is 0. 
since these features represent lengths, so zero values in this columns represent null values.

In [ ]:
print("null values in x column :" , sum(train['x'] == 0))
print("null values in y column :" , sum(train['y'] == 0))
print("null values in z column :" , sum(train['z'] == 0))

In [ ]:
print("number of samples with null values : " , sum((train['x'] == 0) | (train['y'] == 0) | (train['z'] == 0)))

In [ ]:
train[(train['x'] == 0) | (train['y'] == 0) | (train['z'] == 0)]

this is all the samples with null values

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
sns.countplot(x=train.cut,
             order=cut_order)  
plt.xlabel("cut Categories")
plt.ylabel("Counts")
plt.title("distribution of cut categories")
plt.show()

In [ ]:
color_order = ['J' , 'I' , 'H' , 'G' ,'F','E' , 'D']
sns.countplot(x=train.color,
             order = color_order)  
plt.xlabel("color Categories")
plt.ylabel("Counts")
plt.title("distribution of color categories")
plt.show()

In [ ]:
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
sns.countplot(x=train.clarity,
             order = clarity_order)  
plt.xlabel("clarity Categories")
plt.ylabel("Counts")
plt.title("distribution of clarity categories")
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 15))


sns.scatterplot(x='price',
                y='clarity' ,
                hue='clarity',
                data=train ,
                hue_order = clarity_order,
                ax = axes[0])


sns.scatterplot(x='price',
                y='color' ,
                hue='color',
                data=train ,
                hue_order = color_order,
                ax=axes[1])



sns.scatterplot(x='price',
                y='cut' ,
                hue='cut',
                data=train ,
                hue_order = cut_order,
                ax = axes[2])




plt.show()

the previos plot shows the relation between the price and categorical featureas.
there isn't any good or clear relation between price and cut,color,clarity columns

In [ ]:
corr_matrix = train.select_dtypes(include='number').drop('Id' , axis = 1).corr()
corr_matrix

In [ ]:
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')  
plt.title("numerical features Correlations")
plt.show()

In [ ]:
train['size'] = train['x'] * train['y'] * train['z']

In [ ]:
corr_matrix2 = train.select_dtypes(include='number').drop('Id' , axis = 1).corr()
corr_matrix2

In [ ]:
sns.heatmap(corr_matrix2, annot=True, cmap='coolwarm')  
plt.title("numerical features Correlations with size")
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 30))


sns.scatterplot(x='price',
                y='carat' ,
                hue='clarity',
                style = 'clarity',
                data=train ,
                hue_order = clarity_order,
                ax = axes[0])



sns.scatterplot(x='price',
                y='carat' ,
                hue='color',
                style = 'color',
                data=train ,
                hue_order = color_order,
                ax=axes[1])



sns.scatterplot(x='price',
                y='carat' ,
                hue='cut',
                data=train ,
                hue_order = cut_order,
                style = 'cut',
                ax = axes[2])




plt.show()

In [ ]:
sns.scatterplot(x='table',
                y='depth' ,
                
                hue='cut',
                data=train ,
                hue_order = cut_order,
                style='cut',
           )



In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 10))


sns.scatterplot(x='table',
                y='cut' ,
                hue='cut',
                style = 'cut',
                data=train ,
                hue_order = cut_order,
                ax = axes[0])


sns.scatterplot(x='depth',
                y='cut' ,
                hue='cut',
                style = 'cut',
                data=train ,
                hue_order = cut_order,
                ax=axes[1])



plt.show()

In [ ]:
gig, axes = plt.subplots(4, 1, figsize=(10, 20))
sns.scatterplot(x='size',
                y='carat' ,
                data=train ,
                ax=axes[0]
                )
sns.scatterplot(x='x',
                y='carat' ,
                data=train ,
                ax=axes[1]
                )
sns.scatterplot(x='y',
                y='carat' ,
                data=train ,
                ax=axes[2]
                )
sns.scatterplot(x='z',
                y='carat' ,
                data=train ,
                ax=axes[3]
                )


In [ ]:
gig, axes = plt.subplots(3, 1, figsize=(10, 20))
sns.boxplot(x='cut',
                y='price' ,
                order = cut_order,
                showmeans=True,
                data=train ,
                ax=axes[0]
                )
sns.boxplot(x='color',
                y='price' ,
                order = color_order,
                showmeans=True,
                data=train ,
                ax=axes[1]
                )
sns.boxplot(x='clarity',
                y='price' ,
                order = clarity_order,
                showmeans=True,
                data=train ,
                ax=axes[2]
                )


# prepare the data 
<a id="s3"></a>

In [ ]:
def remove_outliers(df , feature):
    q1 = df[feature].quantile(0.25)
    q3 = df[feature].quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 - 3 * iqr
    upper_bound = q3 + 3 * iqr
    
    mask = (df[feature] > lower_bound) & (df[feature] < upper_bound)
    
    return df[mask]

In [ ]:
columns = train.columns
columns

In [ ]:
X_col = [ 'cut', 'color', 'clarity','carat' , 'depth', 'table' ,'x' , 'y' , 'z', 'size']

In [ ]:
for i in X_col[3:]:
    train = remove_outliers(train, i)

In [ ]:
X_train = train[X_col].copy()
y_train = train['price'].copy() 

In [ ]:
X_train.boxplot()

In [ ]:
X_train.shape

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [ ]:
num_attribs = X_col[3:]
cat_attribs = X_col[:3]

num_pipeline = Pipeline([
 ("standardize", MinMaxScaler()),
])

# encoders = {
#     "cut": OrdinalEncoder(categories=[cut_order]),  
#     "color": OrdinalEncoder(categories=[color_order]),
#     "clarity": OrdinalEncoder(categories=[clarity_order]),
# }

transformer = ColumnTransformer(transformers=[
    ("cut", OrdinalEncoder(categories=[cut_order]) , ['cut']),  
    ("color", OrdinalEncoder(categories=[color_order]) , ['color']),
    ("clarity", OrdinalEncoder(categories=[clarity_order]) , ['clarity']),
    ])
cat_pipeline = Pipeline([
#     ('ohe' , OneHotEncoder())
 ('ordinal_encoder' , transformer),
])

full_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", cat_pipeline, cat_attribs),
])

In [ ]:
full_pipeline.fit(X_train)
trans_X_train = full_pipeline.transform(X_train)
trans_X_train

In [ ]:
test['size'] = test['x'] * test['y'] * test['z']
X_test = test[X_col].copy()
y_test = test['price'].copy() 

trans_X_test = full_pipeline.transform(X_test)

# select and train a model
<a id="s4"></a>

## LinearRegression
<a id = 'ln'></a>

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False) 
X_poly = poly.fit_transform(trans_X_train)
x_poly_test = poly.transform(trans_X_test)

In [ ]:
from sklearn.linear_model import LinearRegression

lin_model = LinearRegression()

lin_model.fit(X_poly, y_train)

train_pred = lin_model.predict(X_poly)

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_train, train_pred)
print("Mean squared error:", mse)
print("Root Mean squared error:", np.sqrt(mse))

In [ ]:
test_pred = lin_model.predict(x_poly_test)
test_mse = mean_squared_error(y_test , test_pred)
print("Mean squared error:", test_mse)
print("Root Mean squared error:", np.sqrt(test_mse))

## RandomForestRegressor
<a id = 'rf'></a>

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# forect_model = RandomForestRegressor(n_estimators=100, random_state=42)
# forect_model.fit(trans_X_train, y_train)

# train_pred = forect_model.predict(trans_X_train)

In [ ]:
# mse = mean_squared_error(y_train, train_pred)
# print('Train set')
# print("Mean squared error:", mse)
# print("Root Mean squared error:", np.sqrt(mse))

In [ ]:
# test_pred = forect_model.predict(trans_X_test)
# test_mse = mean_squared_error(y_test, test_pred)
# print('Test set')
# print("Mean squared error:", test_mse)
# print("Root Mean squared error:", np.sqrt(test_mse))

## GradientBoostingRegressor
<a id = 'GBR'></a>

In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor

# GBR_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
# GBR_model.fit(trans_X_train, y_train)

In [ ]:
# train_pred = GBR_model.predict(trans_X_train)
# mse = mean_squared_error(y_train , train_pred )
# print('Train set')
# print("Mean squared error:", mse)
# print("Root Mean squared error:", np.sqrt(mse))

In [ ]:
# test_pred = GBR_model.predict(trans_X_test)
# test_mse = mean_squared_error(y_test , test_pred )
# print('Test set')
# print("Mean squared error:", test_mse)
# print("Root Mean squared error:", np.sqrt(test_mse))

In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1) 
xgb_model.fit(X_poly, y_train)

In [ ]:
train_pred = xgb_model.predict(X_poly)
mse = mean_squared_error(y_train , train_pred )
print('Train set')
print("Mean squared error:", mse)
print("Root Mean squared error:", np.sqrt(mse))

In [ ]:
test_pred = xgb_model.predict(x_poly_test)
test_mse = mean_squared_error(y_test , test_pred )
print('Test set')
print("Mean squared error:", test_mse)
print("Root Mean squared error:", np.sqrt(test_mse))

# fine tune and evaluate the model
<a id="s5"></a>

## fine tune RandomForestregressor
<a id = 'fnrf'></a>

In [ ]:
from sklearn.model_selection import GridSearchCV
# RF_param_grid = {
#     'n_estimators': [200 , 300 , 400 ],
#     'max_depth': [8 , 10 , 15 ],
# }


# RF_grid_search = GridSearchCV(estimator=RandomForestRegressor(), param_grid=RF_param_grid, cv=5, scoring='neg_mean_squared_error')


# RF_grid_search.fit(trans_X_train, y_train)

# RF_best_model = RF_grid_search.best_estimator_
# RF_best_params = RF_grid_search.best_params_
# RF_best_params

In [ ]:
# train_pred = RF_best_model.predict(trans_X_train)
# mse = mean_squared_error(y_train, train_pred)
# print('Train set')
# print("Mean squared error:", mse)
# print("Root Mean squared error:", np.sqrt(mse))

In [ ]:
# test_pred = RF_best_model.predict(trans_X_test)
# test_mse = mean_squared_error(y_test, test_pred)
# print('Test set')
# print("Mean squared error:", test_mse)
# print("Root Mean squared error:", np.sqrt(test_mse))

## fine tune GradientBoostingRegressor
<a id = 'fnGBR'></a>

In [ ]:
# GBR_param_grid = {
#     'n_estimators': [100 , 200 , 300 , 400 ],
#     'max_depth': [8 , 10 , 15 ],
# }


# GBR_grid_search = GridSearchCV(estimator=GradientBoostingRegressor(), param_grid=GBR_param_grid, cv=5, scoring='neg_mean_squared_error')


# GBR_grid_search.fit(trans_X_train, y_train)

# GBR_best_model = GBR_grid_search.best_estimator_
# GBR_best_params = GBR_grid_search.best_params_
# GBR_best_params

In [ ]:
# train_pred = GBR_best_model.predict(trans_X_train)
# mse = mean_squared_error(y_train, train_pred)
# print('Train set')
# print("Mean squared error:", mse)
# print("Root Mean squared error:", np.sqrt(mse))

In [ ]:
# test_pred = GBR_best_model.predict(trans_X_test)
# test_mse = mean_squared_error(y_test, test_pred)
# print('Test set')
# print("Mean squared error:", test_mse)
# print("Root Mean squared error:", np.sqrt(test_mse))

In [ ]:

xgb_param_grid = {
    'n_estimators': [50 , 100 , 150 , 200 , 300 ],
    'learning_rate':[0.1 ,0.2 ,0.3 , 0.5 ]
}


xgb_grid_search = GridSearchCV(estimator=XGBRegressor(), param_grid=xgb_param_grid, cv=5, scoring='neg_mean_squared_error')


xgb_grid_search.fit(trans_X_train, y_train)

xgb_best_model = xgb_grid_search.best_estimator_
xgb_best_params = xgb_grid_search.best_params_
xgb_best_params

In [ ]:
xgb_best_model = xgb_grid_search.best_estimator_
xgb_best_params = xgb_grid_search.best_params_
xgb_best_params

In [ ]:
test_pred = xgb_best_model.predict(trans_X_test)
test_mse = mean_squared_error(y_test , test_pred )
print('Test set')
print("Mean squared error:", test_mse)
print("Root Mean squared error:", np.sqrt(test_mse))

In [ ]:
train_pred = xgb_best_model.predict(trans_X_train)
mse = mean_squared_error(y_train, train_pred)
print('Train set')
print("Mean squared error:", mse)
print("Root Mean squared error:", np.sqrt(mse))

# submit the result
<a id="s6"></a>

In [ ]:
submit_data['size'] = submit_data['x'] * submit_data['y'] * submit_data['z']
X_submit = full_pipeline.transform(submit_data[X_col])
predict_submit = xgb_best_model.predict(X_submit)

In [ ]:
submition = pd.DataFrame()
submition['Id'] = submit_data['Id']
submition['price'] = predict_submit
submition.to_csv('/kaggle/working/submit.csv' , columns = ['Id' , 'price'] , index = False)